In [109]:
import pandas as pd
import requests
from lxml import html
from fake_useragent import UserAgent
from time import sleep
from random import randint

<b>Постановка задачи</b><br>
Спарсить все рестораны Москвы(данные, отзывы, изображения) с сайта restoran.ru

<i>Тест 1</i> - проверка на чувствительность к парсингу

In [11]:
page = requests.get('https://www.restoran.ru/msk/catalog/restaurants/all/') #первая страница каталога ресторанов

In [17]:
page.status_code

200

In [12]:
data = html.fromstring(page.text)

In [13]:
#Из кода сайта мы знаем, что все рестораны хранятся в блоке <div class="priority-list">
#отдельный ресторан хранится в блоке <div class="item">
#отсюда получаем список ресторанов rest_list
rest_list = data.xpath('//div[@class="priority-list"]/div[@class="item"]') 

In [39]:
#На примере первого ресторана в списке, узнаем какие данные мы получили в явном виде
for item in rest_list[0].text_content().split('\n'):
    if item != '':
        print(item.strip())

КМ
Ресторан
Sixty / Сиксти
Забронировать столик
Смотреть на карте
4.5
262 отзыва
В избранное
Реклама
Кухня:
Средиземноморская, Итальянская, Русская
Средний счет :
3000-4000р
Время работы:
пн-вс с 12:00 до последнего гостя
Телефон:
+7 (495) 988-26-56
Адрес:
Москва, Пресненская наб., д. 12
M
Международная (331 м)
Охраняемая платная


Очевидно, что в явном виде получили поля:
<ul>
<li>Название ресторан</li>
<li>Рейтинг</li>
<li>Количество отзывов</li>
<li>Премиумность ресторана(готов ли владельцы платить нам за потенциальных клиентов)</li>
<li>Типы кухни</li>
<li>Средний счет</li>
<li>Время работы</li>
<li>Номер телефона</li>
<li>Адрес</li>
<li>Тип парковки</li>
</ul>
Неплохо конечно, но хотелось бы и линк на ресторан, и фотографии, и отзывы, да и данных побольше. Как с этим быть?
Помимо явных данных, мы можем вытащить и неявные данные. Вот здесь начинается магия!!!<br>
<i>Неявные данные</i> - данные, передаваемые непосредственно в коде сайта и не предназначенные для отображения конечному пользователю, но видимые для поисковых роботов<s> и программистов</s>.<br>
Чтобы получить неявные данные необходимо взглянуть на код сайта, а конкретно в нашем случае, на код внутри элемента div с class="item"

<i>Номер 1</i>
Сразу получаем Широту, Долготу(lon,lat) и ID ресторана на сайте
![T_start](ws1.png)

<i>Номер 2</i> Получаем ссылку на ресторан внутри сайта restoran.ru и, о боги, полный список изображений ресторана(правда, с не очень подходящим для нас размером)
![T_start](ws2.png)

<i>Номер 3</i> Еще немного потусили в коде и уже получили ссылку на отзывы для ресторана
![T_start](ws3.png)

<i>Номер 4</i> Вот и еще данные - рубрикатор по кухне и метро(здесь важна и транслитная часть названий)
![T_start](ws4.png)

Таким образом, мы получили общую картину происходящего и начальный плацдарм для парсинга. Следующий шаг - определение стратегии парсинга

## Стратегия парсинга

Стратегия парсинга определяется из начальных условий задачи:
<ol>
<li>Время на парсинг</li>
<li>Полнота данных</li>
<li>Качество данных</li>
</ol>
Для данного проекта следующие начальные условия: быстрый парсинг(1-3 дня), самые полные данные, очень чистые и красивые данные.

Взглянув на полученную нами картину, мы можем выбрать оптимальную стратегию парсинга.
### 1. Наивный вариант
Пробегаем все страницы каталога и забираем явные не явные данные, получаем: <i>N страниц каталогов* время на одну страницы (1)</i><br>
Стоп, но ведь нам нужны описания и отзывы о ресторанах?
То есть, нам надо зайти на каждую страницу сайта, забрать с нее описание, а затем зайти на каждую страницу отзывов и забрать все отзывы. Как-то не очень эффективно <i>[(формула 1) + (M страниц ресторанов * время на одну страницу) + (K страниц отзывов * время на одну страниц)]</i>, мягко говоря, да и по времени просядем, а еще и IP поблочат за частые обращения к серверу(DDoS не пройдет).<br> 
К сожалению, в рамках данного проекта лайфхаки и фишечки, ускоряющие процесс парсинга не работают и наивный вариант является единственным верным(стохастический варианты и случайные блуждания мы рассматривать не будем, эти концепции не подходят по начальным условиям нашего проекта).

Очевидно, что парсинг разбивается на 3 этапа:
<ol><li>Парсинг каталога</li><li>Парсинг описаний(+можем допарсить какие-нибудь свистелки/перделки)</li><li>Парсинг отзывов</li></ol>
Опять же, очевидно, что это три разных класса. Причем, классы 2 и 3 зависят только от класса 1, а друг от друга не зависят. Таким образом, первым делом мы будем писать класс-парсер 1, т.е. парсинг каталога.

In [145]:
class Catalog():
    def __init__(self):
        self.res_data = pd.DataFrame()
        #self.t_page = 0
        
    def __getRestList(self, page_num):
        t = randint(2, 6)
        print('Delay %s s.' % (t))
        sleep(t)
        page = requests.get('https://www.restoran.ru/msk/catalog/restaurants/all/?page={}'.format(page_num))
        data = html.fromstring(page.text)
        rest_list = data.xpath('//div[@class="priority-list"]/div[@class="item"]') 
        print('-------------CODE %s-------------' % (page.status_code))
        print('-------------%s rests on page %s-------------' % (len(rest_list), page_num))
        return (rest_list, page.status_code)
    
    def __getProp(self, item, prop_name):
        try:
            props = item.cssselect('div.props div.prop')
        except:
            return '-'

        for prop in props:
            try:#проверка наличия свойств у элемента
                #print(prop.cssselect('div.name')[0].text_content().lower())
                if prop_name.lower() in prop.cssselect('div.name')[0].text_content().lower():
                    return prop.cssselect('div.value')[0].text_content().replace('\n','').strip()
            except:
                return '-'
        else:
            return '-'
    
    def __getMetro(self, item):
        try:
            prop = item.cssselect('div.props div.wsubway')[0]
        except:
            return '-'
        try:
            return prop.cssselect('div.value')[0].text_content().replace('\n','').strip()
        except:
            return '-'
    
    def __getDataFromHTML(self, html_data):
        res = {}
        for item in html_data: #changet to html_data
            ID = item.xpath('div')[0].attrib.get('this-rest-id')
            lon = item.xpath('div')[0].attrib.get('lon')
            lat = item.xpath('div')[0].attrib.get('lat')

            ##########.names-start############
            try:
                rest_type = item.cssselect('div.names div.type')[0].text_content().replace('\n','').strip()
            except:
                rest_type = '-'

            try:
                title = item.cssselect('div.names h2 a')[0].text_content().replace('\n','').strip()
                link = item.cssselect('div.names h2 a')[0].attrib.get('href')
            except:
                title = '-'
                link = '-'

            try:
                rating = item.cssselect('div.names div.list-rating-wrap span')[0].text_content().replace('\n','').strip()
            except:
                rating = 0

            try:
                review_count = item.cssselect('div.names div.list-ration-favorite-wrap div')[1].xpath('a')[0].text_content().replace('\n','').strip()
                review_link = item.cssselect('div.names div.list-ration-favorite-wrap div')[1].xpath('a')[0].attrib.get('href')
            except:
                review_count = 0
                review_link = '-'
            ##########.names-end############

            ##########.props-start############
            try:
                if item.cssselect('div.props div.re')[0].text_content() != '':
                    premium = True
                else:
                    premium = False
            except:
                premium = False
            ##########.props-end############
            
            res[ID] = {
                            'title':title,
                            'link':link,
                            'rating':rating,
                            'review_count':review_count,
                            'review_link':review_link,
                            'lon':lon,
                            'lat':lat,
                            'type':rest_type,
                            'premium':premium,
                            'kitchen':self.__getProp(item, 'кухня'),
                            'avrCheck':self.__getProp(item, 'средний счет'),
                            'timing':self.__getProp(item, 'время работы'),
                            'address':self.__getProp(item, 'адрес'),
                            'phone':self.__getProp(item, 'телефон'),
                            'metro':self.__getMetro(item)
                            }

        return pd.DataFrame.from_dict(res, orient='index')

    def run(self, start_page = 0):
        page = start_page
        status_code = 200
        while status_code == 200 and page < 68:
            print('-------------PAGE %s PARSING-------------' % (page))
            data, status_code = self.__getRestList(page)
            parse_data = self.__getDataFromHTML(data)
            self.res_data = self.res_data.append(parse_data)
            print('-------------%s rests append to frame. Shape of frame: %s @ %s-------------' % (parse_data.shape[0], self.res_data.shape[0], self.res_data.shape[1]))
            page += 1

In [146]:
a = Catalog()

In [148]:
%time a.run()

-------------PAGE 0 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 0-------------
-------------20 rests append to frame. Shape of frame: 20 @ 15-------------
-------------PAGE 1 PARSING-------------
Delay 2 s.
-------------CODE 200-------------
-------------20 rests on page 1-------------
-------------20 rests append to frame. Shape of frame: 40 @ 15-------------
-------------PAGE 2 PARSING-------------
Delay 2 s.
-------------CODE 200-------------
-------------20 rests on page 2-------------
-------------20 rests append to frame. Shape of frame: 60 @ 15-------------
-------------PAGE 3 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 3-------------
-------------20 rests append to frame. Shape of frame: 80 @ 15-------------
-------------PAGE 4 PARSING-------------
Delay 4 s.
-------------CODE 200-------------
-------------20 rests on page 4-------------
-------------20 rests append to fram

-------------CODE 200-------------
-------------20 rests on page 39-------------
-------------20 rests append to frame. Shape of frame: 800 @ 15-------------
-------------PAGE 40 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 40-------------
-------------20 rests append to frame. Shape of frame: 820 @ 15-------------
-------------PAGE 41 PARSING-------------
Delay 6 s.
-------------CODE 200-------------
-------------20 rests on page 41-------------
-------------20 rests append to frame. Shape of frame: 840 @ 15-------------
-------------PAGE 42 PARSING-------------
Delay 4 s.
-------------CODE 200-------------
-------------20 rests on page 42-------------
-------------20 rests append to frame. Shape of frame: 860 @ 15-------------
-------------PAGE 43 PARSING-------------
Delay 6 s.
-------------CODE 200-------------
-------------20 rests on page 43-------------
-------------20 rests append to frame. Shape of frame: 880 @ 15------------

In [149]:
catalog = a.res_data.drop_duplicates()

In [150]:
catalog.head()

,phone,lon,rating,timing,lat,title,review_link,address,review_count,link,premium,type,kitchen,metro,avrCheck
1049250,+7 (495) 988-26-56,37.59573242574,4.5,12:00-00:00,55.744328498233,Эларджи,/msk/opinions/restaurants/elardzhi/,"Москва, Гагаринский переулок, д. 15а",40 отзывов,/msk/detailed/restaurants/elardzhi/,True,Ресторан,"Домашняя, Грузинская",Кропоткинская (490 м),1500-2000р
1216052,+7 (495) 988-26-56,37.62332425168423,0.0,12:00–00:00,55.76196428280367,Mr. Lee / Мистер Ли,/bitrix/components/restoran/favorite.add/ajax.php,"Москва, ул. Кузнецкий Мост, д. 7",В избранное,/msk/detailed/restaurants/mr_lee/,True,"Ресторан, Суши-бар","Авторская, Паназиатская, Японская","Кузнецкий мост (72 м), Лубянка (364 м), Театра...",3000-4000р
1232207,+7 (495) 778-89-94,37.545842860372,4.0,12:00-00:00,55.726364613119,Ласточка,/msk/opinions/restaurants/lastochka/,"Москва, Лужнецкая наб., причал «Лужники ...",16 отзывов,/msk/detailed/restaurants/lastochka/,True,"Ресторан, Банкетный зал, Теплоход","Японская, Итальянская, Русская",Воробьевы горы (1.98 км),2000-3000р
1430018,+7 (495) 988-26-56,0,0.0,-,0,La Marée / Ла Маре,/bitrix/components/restoran/favorite.add/ajax.php,-,В избранное,/msk/detailed/restaurants/la_maree/,True,Ресторан,"Рыбная, Средиземноморская","Чеховская, Улица 1905 года, Смоленская (Арбатс...",3000-4000р
1699769,+7 (495) 988-26-56,37.583187628311,4.0,12:00-00:00 (до последнего гостя),55.757730600603,SELFIE / Селфи,/msk/opinions/restaurants/selfie/,"Москва, Новинский бульвар, 31, второй эт ...",20 отзывов,/msk/detailed/restaurants/selfie/,True,Ресторан,"Авторская, Европейская","Баррикадная (355 м), Краснопресненская (466 м)",1500-2000р


Парсинг страниц-каталогов(на слэнге - парсинг листингов) завершен. Теперь необходимо обработать данные, почистить от мусора и палева и привести данные в красивый вид.<br>
Основной методологией определения мусора являются стат. показатели данных. Определяем стат показатели и чистим от них фрейм. Все очень просто!

In [153]:
catalog.premium.value_counts()

False    1300
True       20
Name: premium, dtype: int64

In [154]:
catalog.review_count.value_counts()

В избранное    499
1 отзыв        211
2 отзыва        80
3 отзыва        60
4 отзыва        50
5 отзывов       39
6 отзывов       31
7 отзывов       25
8 отзывов       18
13 отзывов      18
9 отзывов       17
12 отзывов      16
15 отзывов      15
16 отзывов      14
20 отзывов      13
14 отзывов      13
10 отзывов      11
33 отзыва        9
24 отзыва        9
11 отзывов       8
41 отзыв         7
22 отзыва        7
18 отзывов       7
36 отзывов       6
25 отзывов       6
23 отзыва        5
35 отзывов       5
21 отзыв         5
50 отзывов       5
29 отзывов       5
              ... 
63 отзыва        1
188 отзывов      1
93 отзыва        1
44 отзыва        1
34 отзыва        1
88 отзывов       1
131 отзыв        1
53 отзыва        1
101 отзыв        1
79 отзывов       1
64 отзыва        1
60 отзывов       1
116 отзывов      1
179 отзывов      1
92 отзыва        1
83 отзыва        1
126 отзывов      1
130 отзывов      1
76 отзывов       1
49 отзывов       1
107 отзывов      1
139 отзывов 

In [155]:
catalog['review_count'] = catalog.review_count.apply(lambda x: 0 if x.lower() == 'в избранное' else int(x.split(' ')[0])).astype(int)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
catalog.review_count.value_counts()

0      499
1      211
2       80
3       60
4       50
5       39
6       31
7       25
8       18
13      18
9       17
12      16
15      15
16      14
14      13
20      13
10      11
33       9
24       9
11       8
41       7
18       7
22       7
25       6
36       6
35       5
29       5
23       5
21       5
50       5
      ... 
131      1
130      1
134      1
203      1
93       1
188      1
145      1
152      1
139      1
49       1
92       1
88       1
247      1
45       1
44       1
60       1
61       1
63       1
64       1
34       1
76       1
77       1
79       1
81       1
83       1
26       1
85       1
53       1
87       1
86       1
Name: review_count, Length: 92, dtype: int64

In [157]:
catalog.review_link.value_counts()

/bitrix/components/restoran/favorite.add/ajax.php             499
/msk/opinions/restaurants/beluga/                               2
/msk/opinions/restaurants/sixty/                                2
/msk/opinions/restaurants/elardzhi/                             2
/msk/opinions/restaurants/chayka_restoran_yakhta/               2
/msk/opinions/restaurants/azerbaydjan/                          2
/msk/opinions/restaurants/lastochka/                            2
/msk/opinions/restaurants/voronezh/                             2
/msk/opinions/restaurants/oblomov/                              2
/msk/opinions/restaurants/whiterabbit/                          2
/msk/opinions/restaurants/selfie/                               2
/msk/opinions/restaurants/the_mad_cook/                         2
/msk/opinions/restaurants/moregril/                             2
/msk/opinions/restaurants/the-waiters/                          2
/msk/opinions/restaurants/black_thai/                           2
/msk/opini

In [158]:
catalog['review_link'] = catalog.review_link.apply(lambda x: '-' if 'bitrix' in x else x)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [159]:
catalog.review_link.value_counts()

-                                                                      499
/msk/opinions/restaurants/nofar/                                         2
/msk/opinions/restaurants/the_mad_cook/                                  2
/msk/opinions/restaurants/sixty/                                         2
/msk/opinions/restaurants/lastochka/                                     2
/msk/opinions/restaurants/selfie/                                        2
/msk/opinions/restaurants/whiterabbit/                                   2
/msk/opinions/restaurants/elardzhi/                                      2
/msk/opinions/restaurants/beluga/                                        2
/msk/opinions/restaurants/azerbaydjan/                                   2
/msk/opinions/restaurants/voronezh/                                      2
/msk/opinions/restaurants/chayka_restoran_yakhta/                        2
/msk/opinions/restaurants/oblomov/                                       2
/msk/opinions/restaurants

In [160]:
catalog = catalog.reset_index()
catalog['ID'] = catalog['index']
del catalog['index']

In [164]:
catalog['premium'] = False

In [165]:
catalog = catalog.filter(items=['ID', 'title','premium','type', 'kitchen', 'link', 'metro', 'lon','lat','address','phone','rating','timing','review_count','review_link', 'avrCheck'])

In [166]:
catalog = catalog.drop_duplicates()

In [169]:
catalog[catalog.title == 'Бавариус']

,ID,title,premium,type,kitchen,link,metro,lon,lat,address,phone,rating,timing,review_count,review_link,avrCheck
616,386364,Бавариус,False,"Ресторан, Пивной ресторан","Немецкая, Европейская",/msk/detailed/restaurants/bav/,"Маяковская (110 м), Тверская (676 м)",37.59748347138,55.769382848115,"Москва, ул. Тверская 30/2, стр. 1",+7 (495) 699-42-11,3.3,12:00-24:00,12,/msk/opinions/restaurants/bav/,1500-2000р
617,386365,Бавариус,False,"Ресторан, Пивной ресторан","Немецкая, Европейская",/msk/detailed/restaurants/ba/,Фрунзенская (259 м),37.584410349273,55.728107368533,"Москва, Комсомольский пр., д. 21/10",+7 (499) 245-23-95,3.7,12:00-24:00,16,/msk/opinions/restaurants/ba/,1500-2000р


In [170]:
catalog.head()

,ID,title,premium,type,kitchen,link,metro,lon,lat,address,phone,rating,timing,review_count,review_link,avrCheck
0,1049250,Эларджи,False,Ресторан,"Домашняя, Грузинская",/msk/detailed/restaurants/elardzhi/,Кропоткинская (490 м),37.59573242574,55.744328498233,"Москва, Гагаринский переулок, д. 15а",+7 (495) 988-26-56,4.5,12:00-00:00,40,/msk/opinions/restaurants/elardzhi/,1500-2000р
1,1216052,Mr. Lee / Мистер Ли,False,"Ресторан, Суши-бар","Авторская, Паназиатская, Японская",/msk/detailed/restaurants/mr_lee/,"Кузнецкий мост (72 м), Лубянка (364 м), Театра...",37.62332425168423,55.76196428280367,"Москва, ул. Кузнецкий Мост, д. 7",+7 (495) 988-26-56,0.0,12:00–00:00,0,-,3000-4000р
2,1232207,Ласточка,False,"Ресторан, Банкетный зал, Теплоход","Японская, Итальянская, Русская",/msk/detailed/restaurants/lastochka/,Воробьевы горы (1.98 км),37.545842860372,55.726364613119,"Москва, Лужнецкая наб., причал «Лужники ...",+7 (495) 778-89-94,4.0,12:00-00:00,16,/msk/opinions/restaurants/lastochka/,2000-3000р
3,1430018,La Marée / Ла Маре,False,Ресторан,"Рыбная, Средиземноморская",/msk/detailed/restaurants/la_maree/,"Чеховская, Улица 1905 года, Смоленская (Арбатс...",0,0,-,+7 (495) 988-26-56,0.0,-,0,-,3000-4000р
4,1699769,SELFIE / Селфи,False,Ресторан,"Авторская, Европейская",/msk/detailed/restaurants/selfie/,"Баррикадная (355 м), Краснопресненская (466 м)",37.583187628311,55.757730600603,"Москва, Новинский бульвар, 31, второй эт ...",+7 (495) 988-26-56,4.0,12:00-00:00 (до последнего гостя),20,/msk/opinions/restaurants/selfie/,1500-2000р


In [171]:
catalog.to_csv('MskRestCat.csv', index=False, encoding='UTF-8')

Чего не хватает? Описаний и изображений!<br>
Для выгрузки этих данных мы напишем отдельный класс, который пробежит по всем ресторанам и соберет эти данные.